# Load libraries

In [2]:
import h5py
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.layers import LeakyReLU, Dense, Conv1D,Flatten, Dropout, AveragePooling1D, Input, concatenate, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from tensorflow.keras.regularizers import l2

# Helper functions


In [6]:
FILE_PATH = "2018.01/permutated_data.h5"
BUFFER_SIZE = 20000
SAMPLE_SIZE = 851968
BATCH_SIZE = 64
TRAIN_SIZE = int(0.9 * SAMPLE_SIZE)
TEST_SIZE = int(0.05 * SAMPLE_SIZE)
VAL_SIZE = int(0.05 * SAMPLE_SIZE)
NUM_CLASSES = 8
SIGNAL_SIZE = 1024
LEAKY_RELU_ALPHA = 0.1

In [5]:


def split_dataset(dataset, train_size, test_size, validation_size, batch_size):
    '''
    splits a tensorflow dataset into training, validation and test sets
    '''
    train_dataset = dataset.take(train_size//batch_size)
    val_dataset = dataset.skip(train_size//batch_size).take(validation_size//batch_size)
    test_dataset = dataset.skip(train_size//batch_size + validation_size//batch_size).take(test_size//batch_size)

    return train_dataset, val_dataset, test_dataset

def load_data_from_hdf5_amplitude_phase(file_path):
    '''
    loads dataset from hdf5 file and returns a tensorflow datasets splitted
    '''
    def generator():
        with h5py.File(file_path, 'r') as f:
            for i in range(0, SAMPLE_SIZE, BATCH_SIZE):
                batch = f['X'][i:i+BATCH_SIZE]
                batch = tf.cast(batch, tf.float32)
                batch = (batch - tf.reduce_min(batch)) / (tf.reduce_max(batch) - tf.reduce_min(batch))
                i_data = f['X'][i:i+BATCH_SIZE, :, 0]
                q_data = f['X'][i:i+BATCH_SIZE, :, 1]
                complex_data = i_data + 1j*q_data
                
                amplitude = np.abs(complex_data)
                phase = np.angle(complex_data)

                amp_phase_batch = np.stack((amplitude, phase), axis=-1)        
            
                labels = f['Y'][i:i+BATCH_SIZE, :NUM_CLASSES]
                snr = f['Z'][i:i+BATCH_SIZE]
            
                yield batch, amp_phase_batch, labels, snr

    dataset = tf.data.Dataset.from_generator(generator, output_signature=(
    tf.TensorSpec(shape=(BATCH_SIZE, SIGNAL_SIZE, 2), dtype=tf.float32),
    tf.TensorSpec(shape=(BATCH_SIZE, SIGNAL_SIZE, 2), dtype=tf.float32),
    tf.TensorSpec(shape=(BATCH_SIZE, NUM_CLASSES), dtype=tf.int32),
    tf.TensorSpec(shape=(BATCH_SIZE, 1), dtype=tf.float32)
    ))
    
    return split_dataset(dataset, TRAIN_SIZE, TEST_SIZE, VAL_SIZE, BATCH_SIZE)


@tf.autograph.experimental.do_not_convert
def map_dataset(signal, mod, label, snr):
    return ((signal, mod), label)


# MODEL COMPILING

In [ ]:
log_dir = os.path.join(
    "logs",
    "fit",
    datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),
)
# INPUTS
input_iq = Input(shape=(SIGNAL_SIZE, 2))
input_ap = Input(shape=(SIGNAL_SIZE, 2)) 

# I/Q BLOCK

x = Conv1D(64, 5, kernel_regularizer=l2(0.001))(input_iq)
x = LeakyReLU(alpha=LEAKY_RELU_ALPHA)(x)
x = AveragePooling1D(2)(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Conv1D(128, 3, kernel_regularizer=l2(0.001))(x)
x = LeakyReLU(alpha=LEAKY_RELU_ALPHA)(x)
x = AveragePooling1D(2)(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Conv1D(128, 3, kernel_regularizer=l2(0.001))(x)
x = LeakyReLU(alpha=LEAKY_RELU_ALPHA)(x)
x = AveragePooling1D(2)(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Conv1D(256, 3, kernel_regularizer=l2(0.001))(x)
x = LeakyReLU(alpha=LEAKY_RELU_ALPHA)(x)
x = AveragePooling1D(2)(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Conv1D(512, 3, kernel_regularizer=l2(0.001))(x)
x = LeakyReLU(alpha=LEAKY_RELU_ALPHA)(x)
x = AveragePooling1D(2)(x)
x = BatchNormalization()(x)

x = Flatten()(x)
x = Model(inputs=input_iq, outputs=x)

# AMDLITUDE/PHASE BLOCK

y = Conv1D(64, 5, kernel_regularizer=l2(0.001))(input_ap)
y = LeakyReLU(alpha=LEAKY_RELU_ALPHA)(y)
y = AveragePooling1D(2)(y)
y = BatchNormalization()(y)
y = Dropout(0.2)(y)

y = Conv1D(128, 3, kernel_regularizer=l2(0.001))(y)
y = LeakyReLU(alpha=LEAKY_RELU_ALPHA)(y)
y = AveragePooling1D(2)(y)
y = BatchNormalization()(y)
y = Dropout(0.2)(y)

y = Conv1D(128, 3, kernel_regularizer=l2(0.001))(y)
y = LeakyReLU(alpha=LEAKY_RELU_ALPHA)(y)
y = AveragePooling1D(2)(y)
y = BatchNormalization()(y)
y = Dropout(0.2)(y)

y = Conv1D(256, 3, kernel_regularizer=l2(0.001))(y)
y = LeakyReLU(alpha=LEAKY_RELU_ALPHA)(y)
y = AveragePooling1D(2)(y)
y = BatchNormalization()(y)
y = Dropout(0.2)(y)

y = Conv1D(512, 3, kernel_regularizer=l2(0.001))(y)
y = LeakyReLU(alpha=LEAKY_RELU_ALPHA)(y)
y = AveragePooling1D(2)(y)
y = BatchNormalization()(y)

y = Flatten()(y)
y = Model(inputs=input_ap, outputs=y)

# CONCAT BLOCKS
combined = concatenate([x.output, y.output])

# DENSE BLOCK
z = Dense(256, kernel_regularizer=l2(0.001))(combined)
z = LeakyReLU(alpha=LEAKY_RELU_ALPHA)(z)
z = BatchNormalization()(z)
z = Dropout(0.2)(z)

z = Dense(128, kernel_regularizer=l2(0.001))(z)
z = LeakyReLU(alpha=LEAKY_RELU_ALPHA)(z)
z = BatchNormalization()(z)
z = Dropout(0.2)(z)

z = Dense(64, kernel_regularizer=l2(0.001))(z)
z = LeakyReLU(alpha=LEAKY_RELU_ALPHA)(z)
z = BatchNormalization()(z)
z = Dropout(0.2)(z)

z = Dense(32, kernel_regularizer=l2(0.001))(z)
z = LeakyReLU(alpha=LEAKY_RELU_ALPHA)(z)
z = BatchNormalization()(z)
z = Dropout(0.2)(z)

# OUTPUT
z = Dense(NUM_CLASSES, activation="softmax")(z)

model = Model(inputs=[x.input, y.input], outputs=z)

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
checkpoint_callback = ModelCheckpoint(filepath='iq_AP_cnn_50.h5', save_best_only=True, monitor='val_loss', mode='min')
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3, min_delta=0.03)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])

# Training

In [ ]:

train_dataset, val_dataset, test_dataset = load_data_from_hdf5_amplitude_phase(FILE_PATH)

history = model.fit(train_dataset.map(map_dataset), validation_data=val_dataset.map(map_dataset), epochs=50,  callbacks=[tensorboard_callback, checkpoint_callback, early_stopping_callback])